In [ ]:
import pandas as pd
import numpy as np
import datetime

prices = pd.read_csv('/Users/arshvetsoff/Desktop/DIPLOMA/atsenergo_2017_2023_1.csv')
prices['is_mon'] = 0
prices['is_sat'] = 0
prices['is_sun'] = 0
prices.index = prices.timestamp
prices.drop(columns=['timestamp'], inplace=True)
prices.index = pd.to_datetime(prices.index)
#start from 13-03-2020 till 12-03-2021 (365 days)
prices = prices.iloc[28008:28008+12264]
prices = prices.resample('D').mean()
prices['is_mon'] = prices.index.weekday == 0
prices['is_sat'] = prices.index.weekday == 5
prices['is_sun'] = prices.index.weekday == 6


In [ ]:
mins = pd.read_csv('/Users/arshvetsoff/Desktop/DIPLOMA/atsenergo_2017_2023_1.csv')
mins.index = mins.timestamp
mins.drop(columns=['timestamp'], inplace=True)
mins.index = pd.to_datetime(mins.index)
mins = mins.iloc[28008:28008+12264]
mins = mins.resample('D').min()

In [ ]:
temperatures = pd.read_csv('/Users/arshvetsoff/Downloads/temperatures_1.txt', sep='\t')
temperatures['datetime_str'] = temperatures['date'] + ' ' + temperatures['hour']
temperatures['temperature'] = np.log(9/5 * temperatures['temp'] + 32)
temperatures['temperature'].fillna(0, inplace=True)
temperatures['datetime'] = pd.to_datetime(temperatures['datetime_str'], format='%d %B %Y %H:%M')
temperatures.index = temperatures['datetime']
temperatures.drop(columns=['datetime', 'date', 'hour', 'datetime_str', 'temp'], inplace=True)
temperatures = temperatures.resample('D').mean()

In [ ]:
data = pd.merge(prices, mins, right_index=True, left_index=True)
data.head()

In [ ]:
data = pd.merge(data, temperatures, right_index=True, left_index=True)
data['prev_price'] = data['price_y'].shift(1)
data.fillna(1, inplace=True)
data.head()

In [ ]:
train_df = data.iloc[:397]
train_df.index = train_df.index.astype('period[D]')
prices_train = np.log(train_df['price_x'].values.astype(float))
mondays_train = train_df.is_mon.values.astype(int)
saturdays_train = train_df.is_sat.values.astype(int)
sundays_train = train_df.is_sun.values.astype(int)
mins_train = np.log(train_df['prev_price'].values.astype(float))
temperatures_train = train_df.temperature.values.astype(float)

test_df = data.iloc[397:]
test_df.index = test_df.index.astype('period[D]')
prices_test = np.log(test_df['price_x'].values.astype(float))
mondays_test = test_df.is_mon.values.astype(int)
saturdays_test = test_df.is_sat.values.astype(int)
sundays_test = test_df.is_sun.values.astype(int)
mins_test = np.log(test_df['prev_price'].values.astype(float))
temperatures_test = test_df.temperature.values.astype(float)

len(prices_train), len(prices_test)

In [ ]:
train_data = {
    'N' : len(prices_train),
    'y' : list(prices_train),
    'temp' : list(temperatures_train),
    'min_y' : list(mins_train),
    'D_Sat' : list(saturdays_train),
    'D_Sun' : list(sundays_train),
    'D_Mon' : list(mondays_train)
}

test_data = []
for i in range(len(prices_test)):
    test_data.append({
        'y' : prices_test[i],
        'temp' : temperatures_test[i],
        'min_y' : mins_test[i],
        'D_Sat' : saturdays_test[i],
        'D_Sun' : sundays_test[i],
        'D_Mon' : mondays_test[i]
    })

test_data = test_data[:100]

In [ ]:
train_data['min_y'][:5]

In [ ]:
### FIT MODEL
from cmdstanpy import CmdStanModel
model = CmdStanModel(stan_file='/Users/arshvetsoff/Desktop/DIPLOMA/FUCKING_STAN_PREDICTIONS/prod_models/ats_1/temperature_min.stan')

In [ ]:
fit = model.sample(data=train_data, iter_sampling=120000, iter_warmup=800000, chains=1, adapt_delta=0.95)

In [ ]:
fit.save_csvfiles('/Users/arshvetsoff/Desktop/DIPLOMA/FUCKING_STAN_PREDICTIONS/prod_checkpoints/ats_1')

In [ ]:
predicted_values = []

cur_fit = fit.summary().copy().loc[:, 'Mean'].to_dict()
print('copied initial fit')
train_data = {
    'N' : len(prices_train),
    'y' : list(prices_train),
    'temp' : list(temperatures_train),
    'min_y' : list(mins_train),
    'D_Sat' : list(saturdays_train),
    'D_Sun' : list(sundays_train),
    'D_Mon' : list(mondays_train)
}

cnt = 0

for elem in test_data:
    train_data['N'] += 1
    train_data['y'].append(elem['y'])
    train_data['temp'].append(elem['temp'])
    train_data['min_y'].append(elem['min_y'])
    train_data['D_Sat'].append(elem['D_Sat'])
    train_data['D_Sun'].append(elem['D_Sun'])
    train_data['D_Mon'].append(elem['D_Mon'])

    prev_fit = cur_fit.copy()
    print('copied previous fit')
    cur_fit = model.sample(
        data=train_data,
        iter_sampling=8000,
        iter_warmup=5000,
        chains=1,
        inits=prev_fit  # Start from the last parameter means
    )
    predicted_values.append(cur_fit.stan_variable('prediction'))
    cur_fit = cur_fit.summary().loc[:, 'Mean'].to_dict()

In [ ]:
predicted_values